# Import Libraries

In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from art.estimators.classification import TensorFlowV2Classifier
from art.defences.detector.poison import SpectralSignatureDefense

# Load Data

In [18]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [19]:
poisoning_ratio = 0.3

In [20]:
target_classes = [0, 1, 2]

In [21]:
num_poisoned = int(poisoning_ratio * len(y_train))
poison_indices = np.random.choice(len(y_train), num_poisoned, replace=False)

In [22]:
safe_y_train = y_train.copy()
safe_y_test = y_test.copy()

In [23]:
for i in poison_indices:
    if y_train[i] in target_classes:
        y_train[i] = np.random.choice([j for j in range(10) if j != y_train[i]])

In [26]:
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

In [27]:
safe_y_train = to_categorical(safe_y_train, num_classes=10)
safe_y_test = to_categorical(safe_y_test, num_classes=10)

In [28]:
poisoned_y_train = to_categorical(y_train, num_classes=10)
poisoned_y_test = to_categorical(y_test, num_classes=10)

# Model

In [34]:
model = Sequential([
    Input(shape=(32, 32, 3)),
    Conv2D(32, (3, 3), activation="relu", padding="same"),
    Conv2D(32, (3, 3), activation="relu", padding="same"),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(64, (3, 3), activation="relu", padding="same"),
    Conv2D(64, (3, 3), activation="relu", padding="same"),
    MaxPooling2D(pool_size=(2, 2)),
    
    Flatten(),
    
    Dense(512, activation="relu"),
    Dense(10, activation="softmax")
])

In [35]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train

In [40]:
history = model.fit(x_train, safe_y_train, batch_size=128, epochs=10, validation_data=(x_test, safe_y_test))

Epoch 1/10
391/391 [==============================] - 8s 12ms/step - loss: 1.4080 - accuracy: 0.4920 - val_loss: 1.1044 - val_accuracy: 0.6081
Epoch 2/10
391/391 [==============================] - 4s 9ms/step - loss: 0.9580 - accuracy: 0.6631 - val_loss: 0.8873 - val_accuracy: 0.6842
Epoch 3/10
391/391 [==============================] - 4s 9ms/step - loss: 0.7697 - accuracy: 0.7308 - val_loss: 0.8021 - val_accuracy: 0.7222
Epoch 4/10
391/391 [==============================] - 4s 9ms/step - loss: 0.6217 - accuracy: 0.7827 - val_loss: 0.7665 - val_accuracy: 0.7336
Epoch 5/10
391/391 [==============================] - 4s 10ms/step - loss: 0.4930 - accuracy: 0.8265 - val_loss: 0.7562 - val_accuracy: 0.7483
Epoch 6/10
391/391 [==============================] - 4s 10ms/step - loss: 0.3600 - accuracy: 0.8748 - val_loss: 0.7757 - val_accuracy: 0.7583
Epoch 7/10
391/391 [==============================] - 4s 9ms/step - loss: 0.2356 - accuracy: 0.9188 - val_loss: 0.9328 - val_accuracy: 0.7498
Epo

In [44]:
classifier = TensorFlowV2Classifier(model=model, nb_classes=10, input_shape=(32, 32, 3))

# Test

In [59]:
spectral_signature_defense = SpectralSignatureDefense(classifier=classifier, x_train=x_train, y_train=poisoned_y_train)
report, is_clean_lst = spectral_signature_defense.detect_poison(n_components=10)

In [67]:
poisonous_indices = []

# 0 -> Poisoned
# 1 -> Cleaned

for i in range(len(is_clean_lst)):
    if is_clean_lst[i] == 0:
        poisonous_indices.append(i)

In [69]:
poison_indices.size

15000

In [71]:
len(poisonous_indices)

24752